<a href="https://colab.research.google.com/github/olvad/ML_course_openEdu/blob/main/Homeworks/HW_10_ML_COURSE_RECSYS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом задании мы найдем похожие фильмы и пользователей по алгоритму ALS, реализуем подсчет метрики NDCG и исследуем влияние размерности скрытых представлений на работу алгоритма.

Загрузим данные и модели из семинара:

**Важно: не изменяйте код до задания 1!**

In [ ]:
import zipfile
from collections import defaultdict, Counter
import datetime

from scipy import linalg
import numpy as np

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2024-08-01 16:12:09--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  26.7MB/s    in 0.2s    

2024-08-01 16:12:09 (26.7 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [ ]:
# read data
movies = {} # id
users = {} # id
ratings = defaultdict(list) # user-id

with zipfile.ZipFile("ml-1m.zip", "r") as z:
    # parse movies
    with z.open("ml-1m/movies.dat") as m:
        for line in m:
            MovieID, Title, Genres = line.decode('iso-8859-1').strip().split("::")
            MovieID = int(MovieID)
            Genres = Genres.split("|")
            movies[MovieID] = {"Title": Title, "Genres": Genres}

    # parse users
    with z.open("ml-1m/users.dat") as m:
        fields = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]
        for line in m:
            row = list(zip(fields, line.decode('iso-8859-1').strip().split("::")))
            data = dict(row[1:])
            data["Occupation"] = int(data["Occupation"])
            users[int(row[0][1])] = data

    # parse ratings
    with z.open("ml-1m/ratings.dat") as m:
        for line in m:
            UserID, MovieID, Rating, Timestamp = line.decode('iso-8859-1').strip().split("::")
            UserID = int(UserID)
            MovieID = int(MovieID)
            Rating = int(Rating)
            Timestamp = int(Timestamp)
            ratings[UserID].append((MovieID, Rating, datetime.datetime.fromtimestamp(Timestamp)))

In [ ]:
# train-test split
times = []
for user_ratings in ratings.values():
  times.extend([x[2] for x in user_ratings])
times = sorted(times)
threshold_time = times[int(0.8 * len(times))]

train = []
test = []
for user_id, user_ratings in ratings.items():
    train.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] <= threshold_time)
    test.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] > threshold_time)
print("ratings in train:", len(train))
print("ratings in test:", len(test))

ratings in train: 800168
ratings in test: 200041


In [ ]:
train_by_user = defaultdict(list)
test_by_user = defaultdict(list)
for u, i, r in train:
    train_by_user[u].append((i, r))
for u, i, r in test:
    test_by_user[u].append((i, r))

train_by_item = defaultdict(list)
for u, i, r in train:
    train_by_item[i].append((u, r))

n_users = max([e[0] for e in train]) + 1
n_items = max([e[1] for e in train]) + 1

In [ ]:
%%time
# Реализация ALS из семинара
np.random.seed(0)
LATENT_SIZE = 10
N_ITER = 20

# регуляризаторы
lambda_p = 0.2
lambda_q = 0.001

# латентные представления
p = 0.1 * np.random.random((n_users, LATENT_SIZE))
q = 0.1 * np.random.random((n_items, LATENT_SIZE))


def compute_p(p, q, train_by_user):
    for u, rated in train_by_user.items():
        rated_items = [i for i, _ in rated]
        rated_scores = np.array([r for _, r in rated])
        Q = q[rated_items, :]
        A = (Q.T).dot(Q)
        d = (Q.T).dot(rated_scores)
        p[u, :] = np.linalg.solve(lambda_p * len(rated_items) * np.eye(LATENT_SIZE) + A, d)
    return p

def compute_q(p, q, train_by_item):
    for i, rated in train_by_item.items():
        rated_users = [j for j, _ in rated]
        rated_scores = np.array([s for _, s in rated])
        P = p[rated_users, :]
        A = (P.T).dot(P)
        d = (P.T).dot(rated_scores)
        q[i, :] = np.linalg.solve(lambda_q * len(rated_users) * np.eye(LATENT_SIZE) + A, d)
    return q

def train_error_mse(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in train])

def test_error_mse(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in test])


for iter in range(N_ITER):
    p = compute_p(p, q, train_by_user)
    q = compute_q(p, q, train_by_item)

    predictions = p.dot(q.T)

    print(iter, train_error_mse(predictions), test_error_mse(predictions))

0 0.034254066990950016 0.16161048497212951
1 0.030645740984182004 0.15155084906221655
2 0.027045334327151088 0.14384734040494065
3 0.025813288873051232 0.1369731449899051
4 0.025347613143060384 0.13077566964080364
5 0.02509638013540347 0.12524794035311057
6 0.02493404752684068 0.12031008916560125
7 0.02482027996454204 0.11587970123247371
8 0.02473748090535384 0.11188957847429643
9 0.024677350034760324 0.1082859231790354
10 0.024634483994446333 0.10502502426863132
11 0.024604361404763415 0.10207014908552949
12 0.024583346331205864 0.09938950190571325
13 0.024568755099793158 0.09695506282023533
14 0.024558698531058874 0.09474199207447921
15 0.024551877533063843 0.09272824318660171
16 0.02454739123798561 0.09089423607528814
17 0.024544605124752126 0.08922255977615293
18 0.02454306682449277 0.08769769701279093
19 0.024542448316282706 0.08630578168734016
CPU times: user 51.4 s, sys: 27.3 s, total: 1min 18s
Wall time: 1min


## Задание 1

Для фильма "Star Wars: Episode V - The Empire Strikes Back (1980)" найдите 3 самых похожих фильма:
* посчитайте скалярное произведение его эмбеддинга с остальными фильмами;
* найдите максимальные значения - они будут соответствовать ближайшим фильмам;
* вычислите значение id_top1+id_top2+id_top3.

Для решения задания вам пригодится словарь со всеми фильмами `movies`

In [ ]:
for id, film in movies.items():
  for genres, title in film.items():
    if title == 'Star Wars: Episode V - The Empire Strikes Back (1980)':
      index = id

index

1196

In [ ]:
arr = q[index] @ q.T

In [ ]:
text = ('1:', '2:', '3:', '4:')
for num,txt in zip(reversed(sorted(set(arr))),text):
    res = ','.join(map(str,[i for i,n in enumerate(arr) if n == num]))
    print(f'top {txt}  index: {res}')


top 1:  index: 260
top 2:  index: 811
top 3:  index: 1196
top 4:  index: 1420


In [ ]:
260 + 811 + 1196

2267

## Задание 2

Для пользователя с ID=5472:

* Найдите самого похожего, аналогично предыдущему заданию;
* Определите количество фильмов, просмотренных обоими пользователями.

In [ ]:
user = 5472
pdots = [k.dot(p[user]) for k in p]

text = ('1:', '2:', '3:', '4:')
for num,txt in zip(reversed(sorted(set(pdots))),text):
    res = ','.join(map(str,[i for i,n in enumerate(pdots) if n == num]))
    print(f'top {txt}  index: {res}')


top 1:  index: 5072
top 2:  index: 3799
top 3:  index: 1221
top 4:  index: 4565


In [ ]:
np.argmax(pdots)
pdots[5072]

0.07643871184402454

In [ ]:
user1 = ratings[5072]
user2 = ratings[5472]
A = []
B = []
for us in user1:
    A.append(us[0])

for us in user2:
  B.append(us[0])

A, B = np.array(A), np.array(B)

len(np.intersect1d(A, B))

27

## Задание 3

На лекции была рассмотрена метрика для измерения качества работы рекомендательной системы NDCG. Вам необходимо реализовать подсчет DCG и NDCG и вывести значения из клетки ниже; ответ округлите до тысячных.

In [ ]:
import numpy as np

def DCG_k(ratings_list, k):
    '''
    ratings_list: np.array(n_items,)
    k: int
    '''
    dsg = 0

    inde = [x for x, _ in enumerate(ratings_list)]
    rat = np.column_stack((ratings_list, inde))


    cnt = 0
    for i in rat:
        if cnt < k:
            dsg_i = (2 ** i[0] - 1) / np.log2(i[1] + 2)
            dsg += dsg_i
            cnt += 1

    return dsg

def NDCG_k(r, k):
    '''
    ratings_list: np.array(n_items,)
    k: int
    '''

    z = sorted(r, reverse=True)


    ndsg = DCG_k(r, k) / DCG_k(z, k)

    return ndsg


ratings_list = [5, 5, 4, 5, 2, 4, 5, 3, 5, 5, 2, 3, 0, 0, 1, 2, 2, 3, 0]
k = 5
ndcg_value = NDCG_k(ratings_list, k)

print(f"NDCG@{k}: {ndcg_value:.3f}")


NDCG@5: 0.794
